In [380]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
import re
import urllib

---

---

# IMDb Dataframes

## df_akas

In [15]:
df_akas = pd.read_csv('title.akas.tsv', sep='\t')

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [301]:
df_akas.shape

(21248271, 8)

In [304]:
df_akas.iloc[11000000:11000020]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
11000000,tt2694390,4,Épisode datant du 28 décembre 2000,FR,fr,\N,\N,0
11000001,tt2694390,5,Episodio datato 28 dicembre 2000,IT,it,\N,\N,0
11000002,tt2694390,6,28 दिसम्बर 2000 को प्रसारित एपिसोड,IN,hi,\N,\N,0
11000003,tt2694390,7,Folge vom 28. Dezember 2000,DE,de,\N,\N,0
11000004,tt2694392,1,Episódio datado de 29 Dezembro de 2000,PT,pt,\N,\N,0
11000005,tt2694392,2,29 दिसम्बर 2000 को प्रसारित एपिसोड,IN,hi,\N,\N,0
11000006,tt2694392,3,Episodio datato 29 dicembre 2000,IT,it,\N,\N,0
11000007,tt2694392,4,2000年12月29日 のエピソード,JP,ja,\N,\N,0
11000008,tt2694392,5,Folge vom 29. Dezember 2000,DE,de,\N,\N,0
11000009,tt2694392,6,Épisode datant du 29 décembre 2000,FR,fr,\N,\N,0


In [313]:
df_akas.loc[(df_akas['titleId']=='tt6105098')&(df_akas['region']=='US')]
# df_akas.loc[(df_akas['titleId']=='tt4154796')]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
15377391,tt6105098,11,The Lion King,US,\N,imdbDisplay,\N,0


In [314]:
df_akas.loc[(df_akas['title']=='The Sixth Sense')&(df_akas['region']=='US')]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
70233,tt0020389,2,The Sixth Sense,US,\N,imdbDisplay,\N,0
443063,tt0068132,3,The Sixth Sense,US,\N,imdbDisplay,\N,0
768857,tt0167404,35,The Sixth Sense,US,\N,imdbDisplay,\N,0
10925382,tt2647190,1,The Sixth Sense,US,\N,\N,\N,0


In [43]:
df_akas[df_akas['isOriginalTitle']==1]['language'].unique()

array(['\\N', 'ja', 'en', 'da', 'ca'], dtype=object)

In [36]:
df_akas.attributes.iloc[100:150]

100                \N
101                \N
102                \N
103                \N
104    informal title
105                \N
106                \N
107                \N
108                \N
109                \N
110                \N
111                \N
112                \N
113                \N
114                \N
115                \N
116                \N
117                \N
118                \N
119                \N
120                \N
121                \N
122                \N
123                \N
124                \N
125                \N
126                \N
127                \N
128                \N
129                \N
130                \N
131                \N
132                \N
133                \N
134                \N
135                \N
136                \N
137                \N
138                \N
139                \N
140                \N
141                \N
142                \N
143                \N
144                \N
145    com

In [31]:
df_akas.language.value_counts().head(20)

\N     3666753
ja     2517338
fr     2483461
hi     2460515
es     2457683
de     2457278
it     2456550
pt     2456468
en      179241
ru       24591
bg       20036
tr       19421
cmn       7638
sv        7402
he        6320
qbn       4855
sr        3881
yue       3644
fa        2319
ca        2046
Name: language, dtype: int64

In [29]:
df_akas.region.value_counts().head(20)

FR     2624034
JP     2612315
DE     2579594
ES     2563380
IT     2550482
IN     2526109
PT     2504156
US     1020881
\N      506192
GB      262304
CA      123245
XWW     119464
BR       80402
AU       74586
GR       70271
MX       67773
FI       63514
RU       58614
HU       57188
PL       53171
Name: region, dtype: int64

In [58]:
df_akas[df_akas['title'] == 'The Wind'].head(50)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
65179,tt0019529,4,The Wind,\N,\N,\N,\N,0
65534,tt0019585,13,The Wind,US,\N,imdbDisplay,\N,0
65537,tt0019585,16,The Wind,GB,\N,imdbDisplay,\N,0
65553,tt0019585,9,The Wind,\N,\N,original,\N,1
303552,tt0052359,5,The Wind,\N,\N,\N,\N,0
567080,tt0083942,2,The Wind,US,\N,\N,\N,0
645062,tt0094327,5,The Wind,\N,\N,original,\N,1
645066,tt0094327,9,The Wind,US,\N,\N,\N,0
806374,tt0181634,5,The Wind,XWW,en,imdbDisplay,\N,0
941689,tt0240217,1,The Wind,\N,\N,original,\N,1


## df_basics

In [20]:
df_basics = pd.read_csv('title.basics.tsv', sep='\t')

/Users/kevinmacmat/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df_basics.iloc[1500000:1500010]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1500000,tt10928494,tvEpisode,Episode #12.6,Episode #12.6,0,2015,\N,\N,Game-Show
1500001,tt10928496,tvEpisode,Episode #12.8,Episode #12.8,0,2015,\N,\N,Game-Show
1500002,tt10928498,tvEpisode,Episode #12.7,Episode #12.7,0,2015,\N,\N,Game-Show
1500003,tt1092849,tvEpisode,Madden Cheat Ballers,Madden Cheat Ballers,0,2004,\N,\N,"Family,Short"
1500004,tt10928500,tvEpisode,Kitchen Confessions,Kitchen Confessions,0,2018,\N,\N,Short
1500005,tt10928502,tvEpisode,Episode #11.2,Episode #11.2,0,2014,\N,\N,Game-Show
1500006,tt10928504,short,When I'm 20,Khi Tôi 20,0,2008,\N,20,"Drama,Short"
1500007,tt10928506,tvEpisode,Episode #11.3,Episode #11.3,0,2014,\N,\N,Game-Show
1500008,tt10928508,tvEpisode,Episode #1.5251,Episode #1.5251,0,2019,\N,\N,"Drama,Romance"
1500009,tt1092850,tvEpisode,Neverwinter Cheat!,Neverwinter Cheat!,0,2002,\N,\N,"Family,Short"


## df_ratings

In [22]:
df_ratings = pd.read_csv('title.ratings.tsv', sep='\t')

In [57]:
df_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1602
1,tt0000002,6.0,196
2,tt0000003,6.5,1277
3,tt0000004,6.1,121
4,tt0000005,6.1,2039
...,...,...,...
1025270,tt9916576,6.0,9
1025271,tt9916578,8.4,17
1025272,tt9916720,5.6,49
1025273,tt9916766,6.8,13


---

---

# Box Office Mojo Worldwide DF

In [259]:
url = 'https://www.boxofficemojo.com/year/world/'
def get_worldwide(url):
    container = []
    for year in range(1999,2019): #1999, 2019
        df = pd.read_html(url + str(year))
        container.append(df[0])
    return pd.concat(container, axis = 0, ignore_index = True)
df = get_worldwide(url)
df

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%.1
0,1,Star Wars: Episode I - The Phantom Menace,"$924,305,084","$431,088,295",46.6%,"$493,216,789",53.4%
1,2,The Sixth Sense,"$672,806,292","$293,506,292",43.6%,"$379,300,000",56.4%
2,3,Toy Story 2,"$487,059,677","$245,852,179",50.5%,"$241,207,498",49.5%
3,4,The Matrix,"$463,517,383","$171,479,930",37%,"$292,037,453",63%
4,5,Tarzan,"$448,191,819","$171,091,819",38.2%,"$277,100,000",61.8%
...,...,...,...,...,...,...,...
12208,895,Under the Turban,"$1,190","$1,190",100%,-,-
12209,896,Half Brothers,"$1,100","$1,100",100%,-,-
12210,897,Invisible Hands,"$1,034","$1,034",100%,-,-
12211,898,TVTV: Video Revolutionaries,$812,$812,100%,-,-


# Box Office Mojo - Foreign Markets

In [377]:
def get_worldwide_link_list(url):
    container = []
    for year in range(1999,2020): #1999, 2019
        url + str(year)
        container.append(url + str(year))
    return container

# link_list = get_worldwide_link_list(url)
# link_list

['https://www.boxofficemojo.com/year/world/1999',
 'https://www.boxofficemojo.com/year/world/2000',
 'https://www.boxofficemojo.com/year/world/2001',
 'https://www.boxofficemojo.com/year/world/2002',
 'https://www.boxofficemojo.com/year/world/2003',
 'https://www.boxofficemojo.com/year/world/2004',
 'https://www.boxofficemojo.com/year/world/2005',
 'https://www.boxofficemojo.com/year/world/2006',
 'https://www.boxofficemojo.com/year/world/2007',
 'https://www.boxofficemojo.com/year/world/2008',
 'https://www.boxofficemojo.com/year/world/2009',
 'https://www.boxofficemojo.com/year/world/2010',
 'https://www.boxofficemojo.com/year/world/2011',
 'https://www.boxofficemojo.com/year/world/2012',
 'https://www.boxofficemojo.com/year/world/2013',
 'https://www.boxofficemojo.com/year/world/2014',
 'https://www.boxofficemojo.com/year/world/2015',
 'https://www.boxofficemojo.com/year/world/2016',
 'https://www.boxofficemojo.com/year/world/2017',
 'https://www.boxofficemojo.com/year/world/2018',


In [404]:
def get_country_hyperlinks(link_list):    #give it a list
    link_results = []
    for link in link_list:   #use one entry for ex. 2019
        page = requests.get(link) #requests 2019 url
        src = page.content  #turn to page
        soup = BeautifulSoup(src, 'html.parser') #turn 2019 to soup
        all_a = soup.find_all('a') #grab anchors from 2019 soup an put into a list of links
        c = []  
        for x in all_a: #looking through hyper links 'hrefs'
            c.append(x.attrs['href']) #appends to c
            # cleans hyperlinks to only include relevant
        movie_urls_year = []
        for x in c:
            if 'releasegroup' in x:
                movie_urls_year.append(x)
        link_results.extend(movie_urls_year)
    return link_results

# get_region_hyperlinks(link_list)

In [411]:
def get_countries_for_title(url):
    # Make a request for each title
    html_table = urllib.request.urlopen('https://www.boxofficemojo.com/{}'.format(url)).read()

    # Turn into soup
    soop = BeautifulSoup(html_table, "html.parser")
    
    # Grab title
    title = soop.find('h1', class_='a-size-extra-large').get_text()
    
    # Grab countries
    for table in soup.findChildren(attrs={'class': 'a-align-center'}): 
        for c in table.children:
            if c.name in ['tr', 'th']:
                c.unwrap()

    df_market = pd.read_html(str(soop), flavor="bs4") #list of data
    list_dataframe = []
    for dataframe in df_market[1:]:
        list_dataframe.append(dataframe.droplevel(level=0,axis=1))
    result = pd.concat(list_dataframe,ignore_index = True)
    result['title'] = title
    return result

# testing_df = get_countries_for_title('releasegroup/gr3511898629/?ref_=bo_ydw_table_1')

In [415]:
def fill_values(result, result_2):
    a1 = result_2.values[0][0]
    a2 = result_2.values[0][1]
    a3 = result_2.values[0][2]
    a4 = result_2.values[0][3]
    
    act_dict = {
        'Actor 1' : a1,
        'Actor 2' : a2,
        'Actor 3' : a3,
        'Actor 4' : a4
    }

    return pd.concat([result, result_2],axis = 1).fillna(act_dict)

In [414]:
url = 'https://www.boxofficemojo.com/releasegroup/gr3511898629/?ref_=bo_ydw_table_1'
df2 = reroute_twice_and_scrape(url)
pd.concat([testing_df,df2],axis = 1)
# df1['Actor 1'] = 'nuggets'

,Market,Release Date,Opening,Gross,title,0,1,2,3
0,Austria,"Apr 24, 2019","$3,563,042","$8,376,632",Avengers: Endgame,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth
1,Belgium,"Apr 24, 2019","$3,141,578","$8,682,740",Avengers: Endgame,NaN,NaN,NaN,NaN
2,Bulgaria,"Apr 26, 2019","$442,037","$1,469,984",Avengers: Endgame,NaN,NaN,NaN,NaN
3,Croatia,"Apr 25, 2019","$545,732","$1,144,369",Avengers: Endgame,NaN,NaN,NaN,NaN
4,Czech Republic,"Apr 25, 2019","$2,534,713","$6,610,971",Avengers: Endgame,NaN,NaN,NaN,NaN
5,Denmark,"Apr 24, 2019","$4,223,121","$10,254,418",Avengers: Endgame,NaN,NaN,NaN,NaN
6,Egypt,"Apr 26, 2019",–,"$1,754,341",Avengers: Endgame,NaN,NaN,NaN,NaN
7,Finland,"Apr 24, 2019","$2,137,080","$4,938,241",Avengers: Endgame,NaN,NaN,NaN,NaN
8,France,"Apr 24, 2019","$24,971,474","$62,387,891",Avengers: Endgame,NaN,NaN,NaN,NaN
9,Germany,"Apr 24, 2019","$27,108,549","$64,319,294",Avengers: Endgame,NaN,NaN,NaN,NaN


In [412]:
testing_df

,Market,Release Date,Opening,Gross,title
0,Austria,"Apr 24, 2019","$3,563,042","$8,376,632",Avengers: Endgame
1,Belgium,"Apr 24, 2019","$3,141,578","$8,682,740",Avengers: Endgame
2,Bulgaria,"Apr 26, 2019","$442,037","$1,469,984",Avengers: Endgame
3,Croatia,"Apr 25, 2019","$545,732","$1,144,369",Avengers: Endgame
4,Czech Republic,"Apr 25, 2019","$2,534,713","$6,610,971",Avengers: Endgame
5,Denmark,"Apr 24, 2019","$4,223,121","$10,254,418",Avengers: Endgame
6,Egypt,"Apr 26, 2019",–,"$1,754,341",Avengers: Endgame
7,Finland,"Apr 24, 2019","$2,137,080","$4,938,241",Avengers: Endgame
8,France,"Apr 24, 2019","$24,971,474","$62,387,891",Avengers: Endgame
9,Germany,"Apr 24, 2019","$27,108,549","$64,319,294",Avengers: Endgame


In [ ]:
df['Actor 1'] = df[0]
df['Actor 1'] = df[0]
df['Actor 1'] = df[0]

In [262]:
df_market = pd.read_html('https://www.boxofficemojo.com/releasegroup/gr3511898629/?ref_=bo_ydw_table_1')

In [397]:
def scrape_actors(url):
    result = []
    outer_container = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    inner_container = []
    for entry in soup.find('table', id = 'principalCast').findAll('tr')[1:]:
        inner_container.append(entry.get_text()[:-8].split('\n\n')[0])
    result.append(inner_container)
#         result.extend(outer_container)
    return result

In [398]:
def reroute_twice_and_scrape(url):
    '''
    get from release link -> title release -> cast and crew
    '''
    #first click
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    appended = soup.find('a','a-link-normal mojo-title-link refiner-display-highlight').get('href')
    home = 'https://www.boxofficemojo.com/'
    second = '{}{}'.format(home,appended)
    #get the second click
    response2 = requests.get(second)
    soup2 = BeautifulSoup(response2.content)
    appended2 = soup2.find('a',class_='a-size-base a-link-normal mojo-navigation-tab').get('href')
    final_url = home + appended2
    columns = ['Actor 1','Actor 2','Actor 3','Actor 4']
    actor_frame = pd.DataFrame(scrape_actors(final_url))
    return actor_frame


# Final Script

In [418]:
url = 'https://www.boxofficemojo.com/year/world/'
link_list = get_worldwide_link_list(url)
country_list = get_country_hyperlinks(link_list)
for country in country_list:  #get 1 title's hyperlink for all countries
    df1 = get_countries_for_title(country) #returns a dataframe
    df2 = reroute_twice_and_scrape(country) #click twice to get to actors
    final_df = fill_values(df1, df2) # Fills NaN values with actor names and concatenates
    print(final_df)

MissingSchema: Invalid URL '/releasegroup/gr2130858501/?ref_=bo_ydw_table_1': No schema supplied. Perhaps you meant http:///releasegroup/gr2130858501/?ref_=bo_ydw_table_1?

---

# Import for web scraping

In [ ]:
# #/releasegroup/gr3511898629/?ref_=bo_ydw_table_1 Change with list
# html_table = urllib.request.urlopen('https://www.boxofficemojo.com/releasegroup/gr3511898629/?ref_=bo_ydw_table_1').read()

# # fix HTML
# soop = BeautifulSoup(html_table, "html.parser")
# for table in soup.findChildren(attrs={'class': 'a-align-center'}): 
#     for c in table.children:
#         if c.name in ['tr', 'th']:
#             c.unwrap()

# df_market = pd.read_html(str(soop), flavor="bs4")
# df_market_1 = df_market[1]
# df_market_2 = df_market[2]
# df_market_3 = df_market[3]
# df_market_4 = df_market[4]
# new_market = pd.DataFrame(df_market[1:])